# Full Experiment Results Optimization Results

To effectively decide which parameters to use on our models, we have to evaluate the HPO results on the GoEmotions dataset.

In [21]:
import glob
import json
import pandas as pd

results = []
paths = glob.glob('../output/*/*/*.json', recursive=True)
for path in paths:
    with open(path) as fp:
        result_dict = json.load(fp)
        result_tuple = (path, result_dict)
        results.append(result_tuple)

results_df = pd.DataFrame(results, columns=['Path', 'Dict'])

In [22]:
EXPERIMENT_METRICS = ['macro_f1', 'micro_f1']
EXPERIMENT_COLUMNS = ['Dataset', 'ModelType', 'Experiment']
INTERNAL_METRICS = ['precision', 'recall']
WEIGHT_FIELD = 'support'


def parse_path(path):
    dataset, model_type, experiment = path.split('/')[-3:]
    experiment = experiment.split('.')[0]
    return dataset, model_type, experiment


def build_experiment_columns(dataframe):
    path_parts = [parse_path(path) for path in dataframe.Path]
    for i, col in enumerate(EXPERIMENT_COLUMNS):
        dataframe[col] = [parts[i] for parts in path_parts]
    return dataframe


def extract_macro_stats(dataframe, metrics=EXPERIMENT_METRICS):
    split_names = dataframe['Dict'][0]['config']['data_config']['split_names']
    for split in split_names:
        for metric in metrics:
            new_col_name = '{}_{}'.format(split, metric)
            dataframe[new_col_name] = [exp_dict['results'][split][metric] for exp_dict in dataframe['Dict']]
    return dataframe


def extract_category_stats(dataframe, metrics=INTERNAL_METRICS, weight_field=WEIGHT_FIELD):
    split_names = dataframe['Dict'][0]['config']['data_config']['split_names']
    for split in split_names:
        for metric in metrics:
            new_col_name = '{}_{}'.format(split, metric)
            metric_dict = []
            for exp_dict in dataframe['Dict']:
                label_values = list(exp_dict['results'][split]['labels'].values())
                global_weight = sum(label_results[weight_field] for label_results in label_values)
                metric_score = sum(label_results[weight_field] * label_results[metric] / global_weight
                                   for label_results in label_values)
                metric_dict.append(metric_score)
            dataframe[new_col_name] = metric_dict
    return dataframe


def extract_model_details(dataframe):
    dataframe['Extractor'] = [exp_dict['config']['extractor_config']['ex_type'] for exp_dict in dataframe['Dict']]
    dataframe['Model'] = [exp_dict['config']['model_config']['model_name'] for exp_dict in dataframe['Dict']]
    return dataframe


def parse_df(dataframe):
    dataframe = build_experiment_columns(dataframe)
    dataframe = extract_model_details(dataframe)
    dataframe = extract_macro_stats(dataframe)
    dataframe = extract_category_stats(dataframe)
    return dataframe


In [23]:
parsed_df = parse_df(results_df)

parsed_df.sort_values('valid_macro_f1')

,Path,Dict,Dataset,ModelType,Experiment,Extractor,Model,train_macro_f1,train_micro_f1,valid_macro_f1,valid_micro_f1,test_macro_f1,test_micro_f1,train_precision,train_recall,valid_precision,valid_recall,test_precision,test_recall
449,../output/Vent/neural/52ce66a7a5c985ed9d0d9368...,{'config': {'data_config': {'raw_path': 'prepr...,Vent,neural,52ce66a7a5c985ed9d0d9368b83743dd,bert,dnnpool,0.017744,0.029671,0.015231,0.027161,0.015436,0.028582,0.017805,0.511462,0.021659,0.435901,0.016890,0.412615
395,../output/Vent/neural/3d433974ce46ca09abbf59ef...,{'config': {'data_config': {'raw_path': 'prepr...,Vent,neural,3d433974ce46ca09abbf59ef50b23c04,fasttext,dnnpool,0.020536,0.031562,0.017813,0.027809,0.016795,0.027072,0.020763,0.592551,0.022148,0.380454,0.021086,0.247270
402,../output/Vent/neural/cd0b01d095b96f8a7b3f8935...,{'config': {'data_config': {'raw_path': 'prepr...,Vent,neural,cd0b01d095b96f8a7b3f8935c3b125ec,fasttext,dnnpool,0.020063,0.028314,0.017923,0.026142,0.016516,0.026958,0.020251,0.564646,0.019656,0.373224,0.021301,0.231763
415,../output/Vent/neural/1f1c44af1b857830da5e3e72...,{'config': {'data_config': {'raw_path': 'prepr...,Vent,neural,1f1c44af1b857830da5e3e7283e5b698,fasttext,dnnpool,0.020919,0.031359,0.018866,0.028082,0.017998,0.029292,0.021351,0.621047,0.020987,0.348442,0.021385,0.249713
335,../output/Vent/neural/0036bc2fddce4ef1dc6bfc9c...,{'config': {'data_config': {'raw_path': 'prepr...,Vent,neural,0036bc2fddce4ef1dc6bfc9cf84beeb1,fasttext,dnnpool,0.021072,0.030897,0.018890,0.029476,0.017877,0.029394,0.020822,0.630147,0.022004,0.345849,0.022280,0.237447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3613,../output/GoEmotions/neural/812213d5b056aac532...,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,neural,812213d5b056aac532335d8800486c5d,bert,dnnpool,0.593322,0.634571,0.515087,0.559888,0.499480,0.556102,0.609850,0.668141,0.510231,0.642633,0.519813,0.621899
3399,../output/GoEmotions/neural/7e372907c3f585085f...,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,neural,7e372907c3f585085f6d037c3849ca63,bert,dnnpool,0.556554,0.597283,0.515675,0.558934,0.510514,0.555689,0.553376,0.662760,0.514209,0.646395,0.515199,0.634065
4406,../output/GoEmotions/neural/dd9f3603b2210ed1f5...,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,neural,dd9f3603b2210ed1f58e963a99102c59,bert,dnnpool,0.556554,0.597283,0.515675,0.558934,0.510514,0.555689,0.553376,0.662760,0.514209,0.646395,0.515199,0.634065
2182,../output/GoEmotions/neural/f7cacae6393f216913...,{'config': {'data_config': {'raw_path': 'prepr...,GoEmotions,neural,f7cacae6393f2169131c6f7c394e5403,bert,dnnpool,0.564260,0.612822,0.516185,0.562844,0.498440,0.559146,0.580436,0.661801,0.513887,0.652508,0.522387,0.622689


In [24]:
results_df = parsed_df
results_df['DataSource'] = [val['config']['data_config']['cache_path'].split('/')[-2] for val in results_df.Dict]
results_df['Seed'] = [val['config']['seed'] for val in results_df.Dict]
results_df = results_df[results_df.Seed < 5]

final_df = results_df[['Dict', 'Seed', 'DataSource', 'Dataset', 'ModelType', 'Extractor', 'Model', 'test_macro_f1', 'test_micro_f1', 'test_precision', 'test_recall']].round(2)
final_df = final_df.sort_values(['DataSource', 'Extractor', 'Model', 'Seed'])
final_df

,Dict,Seed,DataSource,Dataset,ModelType,Extractor,Model,test_macro_f1,test_micro_f1,test_precision,test_recall
1080,{'config': {'data_config': {'raw_path': 'prepr...,0,GoEmotions-split-cache,GoEmotions,replica,bert,dnnpool,0.48,0.55,0.52,0.61
1063,{'config': {'data_config': {'raw_path': 'prepr...,1,GoEmotions-split-cache,GoEmotions,replica,bert,dnnpool,0.49,0.56,0.51,0.63
1079,{'config': {'data_config': {'raw_path': 'prepr...,2,GoEmotions-split-cache,GoEmotions,replica,bert,dnnpool,0.49,0.55,0.52,0.60
1095,{'config': {'data_config': {'raw_path': 'prepr...,3,GoEmotions-split-cache,GoEmotions,replica,bert,dnnpool,0.48,0.55,0.52,0.61
1050,{'config': {'data_config': {'raw_path': 'prepr...,4,GoEmotions-split-cache,GoEmotions,replica,bert,dnnpool,0.48,0.55,0.52,0.61
...,...,...,...,...,...,...,...,...,...,...,...
241,{'config': {'data_config': {'raw_path': 'prepr...,3,vent-split-robust-random-cache,Vent,replica-full-random,tfidf,rf,0.11,0.13,0.12,0.16
245,{'config': {'data_config': {'raw_path': 'prepr...,0,vent-split-robust-random-cache,Vent,replica-full-random,tfidf,sgd,0.14,0.16,0.14,0.20
243,{'config': {'data_config': {'raw_path': 'prepr...,1,vent-split-robust-random-cache,Vent,replica-full-random,tfidf,sgd,0.14,0.16,0.14,0.20
237,{'config': {'data_config': {'raw_path': 'prepr...,2,vent-split-robust-random-cache,Vent,replica-full-random,tfidf,sgd,0.14,0.16,0.14,0.20


In [25]:
final_df.drop(['Seed'], axis='columns').groupby(['DataSource', 'Extractor', 'Model']).agg(['mean', 'std']).round(2).reset_index()

DataSource Extractor       Model test_macro_f1        \
                                                                  mean   std   
0           GoEmotions-split-cache      bert     dnnpool          0.48  0.01   
1           GoEmotions-split-cache      bert        lstm          0.47  0.00   
2           GoEmotions-split-cache       bow  naivebayes          0.34  0.00   
3           GoEmotions-split-cache       bow          rf          0.45  0.01   
4           GoEmotions-split-cache       bow         sgd          0.45  0.00   
5           GoEmotions-split-cache  fasttext     dnnpool          0.42  0.00   
6           GoEmotions-split-cache  fasttext        lstm          0.44  0.01   
7           GoEmotions-split-cache     tfidf  naivebayes          0.32  0.00   
8           GoEmotions-split-cache     tfidf          rf          0.45  0.01   
9           GoEmotions-split-cache     tfidf         sgd          0.45  0.01   
10                vent-split-cache      bert     dnnpool          0.17  0.00   
11                vent-split-cache      bert        lstm          0.16  0.00   
12                vent-split-cache       bow  naivebayes          0.11  0.00   
13                vent-split-cache       bow          rf          0.13  0.00   
14                vent-split-cache       bow         sgd          0.13  0.00   
15                vent-split-cache  fasttext     dnnpool          0.12  0.00   
16                vent-split-cache  fasttext        lstm          0.03  0.00   
17                vent-split-cache     tfidf  naivebayes          0.11  0.00   
18                vent-split-cache     tfidf      random          0.02  0.00   
19                vent-split-cache     tfidf          rf          0.13  0.00   
20                vent-split-cache     tfidf         sgd          0.14  0.00   
21         vent-split-random-cache     tfidf      random          0.02  0.00   
22         vent-split-robust-cache       bow  naivebayes          0.13  0.00   
23         vent-split-robust-cache       bow          rf          0.11  0.00   
24         vent-split-robust-cache       bow         sgd          0.13  0.00   
25         vent-split-robust-cache  fasttext     dnnpool          0.13   NaN   
26         vent-split-robust-cache  fasttext        lstm          0.15   NaN   
27         vent-split-robust-cache     tfidf  naivebayes          0.14  0.00   
28         vent-split-robust-cache     tfidf      random          0.02  0.00   
29         vent-split-robust-cache     tfidf          rf          0.11  0.00   
30  vent-split-robust-random-cache       bow  naivebayes          0.13  0.00   
31  vent-split-robust-random-cache       bow          rf          0.11  0.00   
32  vent-split-robust-random-cache     tfidf  naivebayes          0.14  0.00   
33  vent-split-robust-random-cache     tfidf      random          0.02  0.00   
34  vent-split-robust-random-cache     tfidf          rf          0.11  0.00   
35  vent-split-robust-random-cache     tfidf         sgd          0.14  0.00   

   test_micro_f1       test_precision       test_recall        
            mean   std           mean   std        mean   std  
0           0.55  0.00           0.52  0.00        0.61  0.01  
1           0.57  0.00           0.53  0.01        0.62  0.01  
2           0.46  0.00           0.43  0.00        0.52  0.00  
3           0.52  0.00           0.50  0.00        0.59  0.00  
4           0.53  0.00           0.48  0.00        0.61  0.00  
5           0.49  0.00           0.45  0.01        0.61  0.01  
6           0.54  0.00           0.51  0.00        0.58  0.01  
7           0.43  0.00           0.42  0.00        0.48  0.00  
8           0.52  0.00           0.48  0.00        0.60  0.00  
9           0.53  0.00           0.49  0.00        0.59  0.00  
10          0.18  0.00           0.16  0.00        0.24  0.00  
11          0.18  0.00           0.17  0.01        0.23  0.00  
12          0.13  0.00           0.10  0.00        0.19  0.00  
13          0.14  0.00 